**The following file is to calibrate and validate the model with 4 years of data with the results from spinup as the initial condition. Previously in the folder rk_model_final - /home/rk/ats_rk/testing/ats-demos/rk_model/Spinup_final_16032022. The calibration was carried out for only 1 year. All the files will have a similar naming with the ending cv (calibration and validation) and su (spinup).**

* The notebook is developed to code the methodology and extract relevant files that are suitable as input to PEST.
- pestpp/pestpp/benchmarks/mf6_freyberg - This is used as the standard benchmark folder from which the files are referred.
- Currently, the freyberg6_run_glm.pst which solves the Levenberg - Marquardt algorithm is considered.

Parameters considered: 'alpha_p_gp', 'alpha_m_gp', 'n_p_gp', 'n_m_gp', 'wr_p_gp', 'wr_m_gp', 'tcs_p_gp', 'tcs_m_gp', 'tcd_p_gp', 'tcd_m_gp', 'af_p_gp', 'af_m_gp', 'auf_p_gp', 'auf_m_gp', 'por_p_gp', 'por_m_gp', 'perm_p_gp', 'perm_m_gp', 'den_p_gp', 'den_m_gp'

- Note that the bottom B.C is also added to the input file and it will be added later on as a parameter

- The initial conditions are taken from the folder - //home/rk/ats_rk/testing/ats-demos/rk_model/Spinup_final_16032022/Case1_B_cv_spinup_II_final_HD1.demo/checkpoint_final.h5

- We are considering the initial parameters based on the simulation : all_params_afterrw.demo

- The lower and upper bound is fixed to 75 % of the initial value (for most of the parameters, some parameters have varied between a certain range since simulations do not run)



* Write down the steps followed in the notebook

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
%matplotlib inline
import sys,os
import colors
import numpy as np
import matplotlib.cm
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import h5py
import pandas as pd
from datetime import datetime
import matplotlib.image as mpimg

In [3]:
import shutil
import numpy as np
import pandas as pd
import pyemu
import flopy
import subprocess

In [7]:
directory_name = 'Case1_su_cv_start.demo' # Main directory name where the data is stored for the initial round of calibration
file_name = 'Case1_su_cv' # Name of the xml file

/home/rk/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/rk/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 1. Observation data

In [10]:
df_obs_AWS = pd.read_csv('Final_OutputData_4yrs/Calib_outputdata_2016_2018.csv',sep='\t')
df_obs_AWS.head()

# The observation data was processed previously in /home/rk/ats_rk/testing/ats-demos/rk_model/Data/Data_Yakou/Yakou_met_data_ITP_rk/AWS_final_metdata_4yrs_concise.ipynb

,TIMESTAMP,Soil moisture at - 0.04 m (%),Soil moisture at - 0.1 m (%),Soil moisture at - 0.2 m (%),Soil moisture at - 0.4 m (%),Soil moisture at - 0.8 m (%),Soil moisture at - 1.2 m (%),Soil moisture at - 1.6 m (%),Soil temperature at - 0.00 m (°C),Soil temperature at - 0.04 m (°C),Soil temperature at - 0.1 m (°C),Soil temperature at - 0.2 m (°C),Soil temperature at - 0.4 m (°C),Soil temperature at - 0.8 m (°C),Soil temperature at - 1.2 m (°C),Soil temperature at - 1.6 m (°C)
0,2016-01-01,7.897694,7.523868,8.694236,9.016111,5.368792,4.093549,6.058042,-13.103958,-12.507847,-11.848819,-11.295486,-9.824722,-7.814958,-4.874889,-3.447965
1,2016-01-02,7.991826,7.594250,8.745417,9.012639,5.350889,4.060958,6.008417,-12.255417,-11.861597,-11.342569,-10.947708,-9.781389,-7.936854,-5.063889,-3.639549
2,2016-01-03,7.992500,7.606875,8.766806,9.033472,5.342833,4.033590,5.962187,-12.737500,-11.824514,-11.200694,-10.765278,-9.618403,-7.946111,-5.228028,-3.817500
3,2016-01-04,7.960069,7.581465,8.749236,9.025833,5.336757,4.012583,5.923819,-13.006597,-11.997847,-11.365486,-10.867778,-9.648889,-7.977951,-5.358771,-3.977958
4,2016-01-05,7.913792,7.539000,8.703056,8.995694,5.324597,3.993715,5.892174,-13.525278,-12.371597,-11.691181,-11.171181,-9.829653,-8.078194,-5.476889,-4.119438


In [11]:
len(df_obs_AWS)

1096

In [12]:
# Calculating the temperature in Kelvin

depths = [0.04, 0.1, 0.2, 0.4, 0.8, 1.2, 1.6]

time_values = 1096 # Number of time values

times = np.arange(1, time_values+1, 1)

# Adding column information:


x = 0
for i, depth in enumerate(depths):
        df_obs_AWS[f'Soil temperature at - {depth} m (K)'] = df_obs_AWS[f'Soil temperature at - {depth} m (°C)'] + 273.15

df_obs_AWS['Soil temperature at - 0.04 m (K)'].head()

0    260.642153
1    261.288403
2    261.325486
3    261.152153
4    260.778403
Name: Soil temperature at - 0.04 m (K), dtype: float64

In [13]:
# Extracting an example 
obs_data_example = pd.read_csv('Freyberg_example/freyberg6_run_glm.obs_data.csv') 
obs_data_example.head()

,obsnme,obsval,weight,obgnme
0,gage_1_20151231,951.71,0.000000,gage
1,gage_1_20160131,1530.10,0.004357,gage
2,gage_1_20160229,1855.30,0.003593,gage
3,gage_1_20160331,1907.10,0.003496,gage
4,gage_1_20160430,1747.70,0.003815,gage


The following details need to be defined:

Note: Here we need to add the values from the excel that we read previously.

- OBSNME: stemp_{depth}_{1-1096} & smois_{depth}_{1-1096}: Observation names - We have temperature and moisture values at 7 depths. 
- OBSVAL: The corresponding values need to be added in pandas from the dataframe - data_AWS_syn_2017
- WEIGHT (Initial): Assigning equal weights to all variables. 1/15344 = 6.517e-05; 2 {soil_temp, soil_mois} * 7 {7 sensors} * 1096 {1-365}  = 15344
- obgnme: 'temp', 'mois'

In [14]:
# Initial values 
time_values = 1096 # Number of time values
# Total number of values 1/15344 = 6.517e-05; 2 {soil_temp, soil_mois} * 7 {7 sensors} * 1096 {1-365}  = 15344
total_values = 2 * 7 * 1096 
ini_wt = 1/total_values # Initial weight
ini_wt

6.517205422314912e-05

In [15]:
# Creating a new dataframes with columns = obs_data_example.index and rows = 0 - 15344
# There are 1096 values for one sensor in each year, hence we move from 0 - 1095 & then it repeats again for the next sensor.

obs_data = pd.DataFrame(data=None,columns=obs_data_example.columns,index=np.arange(1,total_values+1,1))


depths = [0.04, 0.1, 0.2, 0.4, 0.8, 1.2, 1.6]

times = np.arange(1, time_values+1, 1)

# Adding column information:


x = 0
for i, depth in enumerate(depths):
    for j, time in enumerate(times):
        # Column name = obsnme
        obs_data.iloc[x,0] = f'stemp_{depth}_{j}'
        # Column name = obsval
        obs_data.iloc[x,1] = df_obs_AWS[f'Soil temperature at - {depth} m (K)'][j]
        # Column name = weights : Assigning equal weight to all variables
        obs_data.iloc[x,2] = 1/total_values
        # Column name = obgnme : Assigning observation group name
        obs_data.iloc[x,3] = 'temp'
        x = x + 1

for i, depth in enumerate(depths):
    for j, time in enumerate(times):
        obs_data.iloc[x,0] = f'smois_{depth}_{j}'
        # Column name = obsval
        obs_data.iloc[x,1] = df_obs_AWS[f'Soil moisture at - {depth} m (%)'][j]
        # Column name = weights : Assigning equal weight to all variables
        obs_data.iloc[x,2] = 1/total_values
        # Column name = obgnme : Assigning observation group name
        obs_data.iloc[x,3] = 'mois'
        x = x + 1

        
obs_data

,obsnme,obsval,weight,obgnme
1,stemp_0.04_0,260.642153,0.000065,temp
2,stemp_0.04_1,261.288403,0.000065,temp
3,stemp_0.04_2,261.325486,0.000065,temp
4,stemp_0.04_3,261.152153,0.000065,temp
5,stemp_0.04_4,260.778403,0.000065,temp
...,...,...,...,...
15340,smois_1.6_1091,4.602889,0.000065,mois
15341,smois_1.6_1092,4.57275,0.000065,mois
15342,smois_1.6_1093,4.542944,0.000065,mois
15343,smois_1.6_1094,4.513437,0.000065,mois


In [16]:
df_obs_AWS['Soil temperature at - 0.2 m (K)'][808:812]

808    266.270562
809    265.665708
810    265.340674
811    265.065896
Name: Soil temperature at - 0.2 m (K), dtype: float64

In [17]:
# Check the observation values
obs_data.iloc[3000:3005]

,obsnme,obsval,weight,obgnme
3001,stemp_0.2_808,266.270562,0.000065,temp
3002,stemp_0.2_809,265.665708,0.000065,temp
3003,stemp_0.2_810,265.340674,0.000065,temp
3004,stemp_0.2_811,265.065896,0.000065,temp
3005,stemp_0.2_812,265.209056,0.000065,temp


In [18]:
directory_name

'Case1_su_cv_start.demo'

In [19]:
# Exporting the parameter group csv file
obs_data.to_csv(f'{directory_name}/{file_name}_obs_data.csv', index=False)

In [20]:
print(f'{file_name}_obs_data.csv')

Case1_su_cv_obs_data.csv


### 2. Parameter groups external
- Extracting the data from the example

In [21]:
pargrp_data_example = pd.read_csv('Freyberg_example/freyberg6_run_glm.pargrp_data.csv') 
pargrp_data_example.head()

,pargpnme,inctyp,derinc,derinclb,forcen,derincmul,dermthd,splitthresh,splitreldiff,splitaction
0,sto_ss_0,relative,0.01,0.0,switch,2.0,parabolic,0.00001,0.5,smaller
1,npf_k33_1,relative,0.01,0.0,switch,2.0,parabolic,0.00001,0.5,smaller
2,npf_k_1,relative,0.01,0.0,switch,2.0,parabolic,0.00001,0.5,smaller
3,sto_ss_1,relative,0.01,0.0,switch,2.0,parabolic,0.00001,0.5,smaller
4,npf_k_0,relative,0.01,0.0,switch,2.0,parabolic,0.00001,0.5,smaller


#### Defining the variables in Parameter Group variables: 
- PARGPNME : Individual groups assigned for all parameters! - Assigned a total of 20 parameters (10 for each material)
- INCTYP : 'relative' - The increment used for forward-difference calculation of derivatives with respect to any parameter belonging to the group is calculated as the fraction of the current value of that parameter; that fraction is provided as the real variable DERINC. Ex: If current value of parameter = 10 & DERINC =  0.01. Then the next parameter value = 0.01 * 10 + 10 = 0.1 + 10.
- DERINC : '0.01' - The fraction of increment of the current value of parameter. [Consider the range of variation of the parameters, upper and lower bounds]
- DERINCLB : '0.0'
- FORCEN : 'switch'. In the first iteration, forward difference method is employed. From the second iteration, it switches to central difference method for the remainder of the inversion process on the iteration after which the relative objective function reduction between successive iterations is less than PHIREDSWH. Note we need to define PHIREDSWH (Where is it defined in this format?)
- DERINCMUL : '1.0' - If three-point derivatives calculation is employed, the value of DERINC is multiplied by DERINCMUL.
- DERMTHD : 'parabolic' - This is preferred as it provides greater accuracy.
- [SPLITTHRESH] [SPLITRELDIFF] [SPLITACTION] - For the first analysis, we ignore the three variables.

In [24]:
pargrp = pargrp_data_example.copy()
# Dropping all rows to replace with the two newly defined rows
pargrp.drop(pargrp.index, inplace=True)

# Dropping the columns - splitthresh, splitreldiff, splitaction
pargrp.drop(columns=['splitthresh', 'splitreldiff', 'splitaction'], inplace=True)

group_names = ['alpha_p_gp', 'alpha_m_gp', 'n_p_gp', 'n_m_gp', 'wr_p_gp', 'wr_m_gp', 'tcs_p_gp', 'tcs_m_gp', 'tcd_p_gp', 'tcd_m_gp', 'af_p_gp', 'af_m_gp', 'auf_p_gp', 'auf_m_gp', 'por_p_gp', 'por_m_gp', 'perm_p_gp', 'perm_m_gp', 'den_p_gp', 'den_m_gp','b_bc_gp']

for i, gp_nm in enumerate(group_names):
    pargrp.loc[i] = [gp_nm, 'relative',0.1,0.0,'switch',1.0,'parabolic']
    #print(i, gp_nm)
    
pargrp.head()

,pargpnme,inctyp,derinc,derinclb,forcen,derincmul,dermthd
0,alpha_p_gp,relative,0.1,0.0,switch,1.0,parabolic
1,alpha_m_gp,relative,0.1,0.0,switch,1.0,parabolic
2,n_p_gp,relative,0.1,0.0,switch,1.0,parabolic
3,n_m_gp,relative,0.1,0.0,switch,1.0,parabolic
4,wr_p_gp,relative,0.1,0.0,switch,1.0,parabolic


In [25]:
directory_name

'Case1_su_cv_start.demo'

In [77]:
### The derinc value for bottom_bc needs to be reduced by a factor of 10 - i.e. derinc = 0.001
# 268.45 * 0.1 = 26.845 which does not lie in the parameter range (263.15 - 274.15)

pargrp.iloc[20,2] = 0.01

In [26]:
# Exporting the parameter group csv file
pargrp.to_csv(f'{directory_name}/{file_name}_pargrp_data.csv', index=False)

### 3. Parameter names external
There are 20 parameters that we are considering for this first analysis (Not considering latent heat and heat capacity)

In [30]:
par_data_example = pd.read_csv('Freyberg_example/freyberg6_run_glm.par_data.csv') 
par_data_example.head()

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom,partied
0,npf_k33_0_000_000,log,factor,0.3,0.03,3.0,npf_k33_0,1.0,0.0,1,NaN
1,npf_k33_0_000_001,tied,factor,0.3,0.03,3.0,npf_k33_0,1.0,0.0,1,npf_k33_0_000_000
2,npf_k33_0_000_002,tied,factor,0.3,0.03,3.0,npf_k33_0,1.0,0.0,1,npf_k33_0_000_000
3,npf_k33_0_000_003,tied,factor,0.3,0.03,3.0,npf_k33_0,1.0,0.0,1,npf_k33_0_000_000
4,npf_k33_0_000_004,tied,factor,0.3,0.03,3.0,npf_k33_0,1.0,0.0,1,npf_k33_0_000_000


The following details need to be defined:
- PARNME: 'n_m', 'tcs_m', 'af_m', 'af_p', .... - It is the parameter name
- PARTRANS: 'none' {'log' - Can we considered later if the inversion process does not occur. Log-transformations helps in ensuring that the parameter changes and model output changes are more linear.}
- PARCHGLIM: 'factor' - Alteration to a parameter's value is factor-limited. { PARCHGLIM must be provided with a value of “relative” or “factor”. The former designates that alterations to a parameter’s value are factor-limited whereas the latter designates that alterations to its value are relative-limited. }
- PARVALI: 1.2, 1, 0.05, 0.005 - These are the starting values for the parameters
- PARLBND: 1.05, 0.8, 0.02, 0.002 - Lower bounds for the parameters.
- PARUBND: 3, 2.5, 0.1, 0.01 - Upper bounds for the parameters.
- PARGRP: It is the parameter group names associated with the parameters. 
- SCALE: 1.0, 1.0, 1.0, 1.0 - No scale or offset is provided
- OFFSET: 0.0, 0.0, 0.0, 0.0 - No scale or offset is provided
- DERCOM: 1, 1, 1, 1 - Only model command exists. Hence we give 1 which represents 'ats'
- Partied: This column will be dropped since we have no tied elements

In [33]:
# Getting the best parameter dataset from the simulation - all_params_afterrw.demo
directory_name_params = 'all_params_afterrw.demo'
best_params_afterrw = pd.read_csv(f'{directory_name_params}/rk_model_glm_cf_v1_allparams_rw.par',delimiter='\s+',skiprows=1, header=None)
best_params_afterrw.drop(columns=[2,3],inplace=True)
best_params_afterrw.columns = ['parameter','PARVAL1']
best_params_afterrw.head()



length_index = len(best_params_afterrw.index)


best_params_afterrw.index = best_params_afterrw['parameter']
best_params_afterrw.drop(columns='parameter',inplace=True)


# Changing the initial value of n_p, n_m and den_m
best_params_afterrw.loc['n_p'] = 2.05 # Since value below 1.05 is not allowed
best_params_afterrw.loc['n_m'] = 2.05 # Since value below 1.05 is not allowed
best_params_afterrw.loc['den_m'] = 1900 # Since value above 2700 represents a soil mineral layer
best_params_afterrw.loc['b_bc'] = 268.45 # Average temperature at Yakou catchment - -4.7 °C

#best_params_afterrw.loc['perm_p'] = 5.346622127220000e-11

# Changing to float type 
best_params_afterrw = best_params_afterrw.astype('float64')

# Adding the lower and upper bound - 75% for all params except n_p, n_m and den_m which is given 50 %
# The bottom B.C - upper bound is given 1°C (274.15) and the lower bound is given -10°C (263.15)
best_params_afterrw['PARLBND'] = best_params_afterrw['PARVAL1']*0.25
best_params_afterrw['PARUBND'] = best_params_afterrw['PARVAL1']*1.75

best_params_afterrw.loc['n_p']['PARLBND'] = best_params_afterrw.loc['n_p']['PARVAL1']*0.5
best_params_afterrw.loc['n_p']['PARUBND'] = best_params_afterrw.loc['n_p']['PARVAL1']*1.5

best_params_afterrw.loc['n_m']['PARLBND'] = best_params_afterrw.loc['n_m']['PARVAL1']*0.5
best_params_afterrw.loc['n_m']['PARUBND'] = best_params_afterrw.loc['n_m']['PARVAL1']*1.5

best_params_afterrw.loc['den_m']['PARLBND'] = best_params_afterrw.loc['den_m']['PARVAL1']*0.5
best_params_afterrw.loc['den_m']['PARUBND'] = best_params_afterrw.loc['den_m']['PARVAL1']*1.5

# The bottom B.C - upper bound is given 1°C (274.15) and the lower bound is given -10°C (263.15)
best_params_afterrw.loc['b_bc']['PARLBND'] = 263.15 
best_params_afterrw.loc['b_bc']['PARUBND'] = 274.15

best_params_afterrw.head()

,PARVAL1,PARLBND,PARUBND
parameter,,,
alpha_p,0.010000,0.002500,0.017500
alpha_m,0.001825,0.000456,0.003194
n_p,2.050000,1.025000,3.075000
n_m,2.050000,1.025000,3.075000
wr_p,0.224324,0.056081,0.392567


In [36]:
best_params_afterrw.iloc[-1,:]

PARVAL1    268.45
PARLBND    263.15
PARUBND    274.15
Name: bottom_bc, dtype: float64

* On checking out the outputs of changing the bottom B.C, it does not impact the results by much!

In [38]:
par_data = par_data_example.copy()
par_data.drop(par_data.index, inplace=True)
# Dropping the columns - splitthresh, splitreldiff, splitaction
par_data.drop(columns=['partied'], inplace=True)

#param_names = ['alpha_p', 'alpha_m', 'n_p', 'n_m', 'wr_p', 'wr_m', 'tcs_p', 'tcs_m', 'tcd_p', 'tcd_m', 'af_p', 'af_m', 'auf_p', 'por_p', 'por_m', 'perm_p', 'perm_m', 'den_p', 'den_m', 'lh_w','lh_i','hc_l','hc_r']

for i, values in enumerate(best_params_afterrw.index):
    par_data.loc[i] = [best_params_afterrw.index[i], 'none','factor', best_params_afterrw['PARVAL1'].values[i], best_params_afterrw['PARLBND'].values[i], best_params_afterrw['PARUBND'].values[i], pargrp['pargpnme'].values[i], 1.0, 0.0, 1] 

par_data.head()
# Adding row information:
#par_data.loc[0] = ['alpha_p','none','factor', 0.05, 0.01, 0.1, 'alpha_p_gp', 1.0, 0.0, 1]
#par_data.loc[1] = ['alpha_m','none','factor', 0.005, 0.001, 0.01, 'alpha_m_gp', 1.0, 0.0, 1]

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
0,alpha_p,none,factor,0.010000,0.002500,0.017500,alpha_p_gp,1.0,0.0,1
1,alpha_m,none,factor,0.001825,0.000456,0.003194,alpha_m_gp,1.0,0.0,1
2,n_p,none,factor,2.050000,1.025000,3.075000,n_p_gp,1.0,0.0,1
3,n_m,none,factor,2.050000,1.025000,3.075000,n_m_gp,1.0,0.0,1
4,wr_p,none,factor,0.224324,0.056081,0.392567,wr_p_gp,1.0,0.0,1


In [39]:
# Exporting the parameter group csv file
par_data.to_csv(f'{directory_name}/{file_name}_par_data.csv', index=False)

### Template files

- It could be created by manually editing the files.

### Instruction files


In [40]:
os.getcwd()

'/home/rk/pestpp/pestpp/rk_model_final_4yrs_su'

In [41]:
directory_name

'Case1_su_cv_start.demo'

In [42]:
# Reading the simulated data - from the xml file generated
sim_data = pd.read_csv(f'{directory_name}/observation.dat',sep=' ',skiprows=88) 

sim_data.head()

,time [s],point -0.04 temperature [K],point -0.1 temperature [K],point -0.2 temperature [K],point -0.4 temperature [K],point -0.8 temperature [K],point -1.2 temperature [K],point -1.6 temperature [K],point -0.04 saturation liquid,point -0.1 saturation liquid,point -0.2 saturation liquid,point -0.4 saturation liquid,point -0.8 saturation liquid,point -1.2 saturation liquid,point -1.6 saturation liquid
0,0.0,270.150000,270.150000,270.150000,270.150000,270.150000,270.150000,270.150000,0.164877,0.164877,0.164877,0.165884,0.165884,0.165884,0.165884
1,86400.0,259.136901,262.535868,265.147149,267.394073,269.696063,270.095452,270.141342,0.164562,0.164595,0.164637,0.165872,0.165881,0.165883,0.165884
2,172800.0,258.197106,260.855646,263.138526,265.497559,268.847926,269.857438,270.071454,0.164555,0.164576,0.164603,0.165869,0.165876,0.165882,0.165883
3,259200.0,257.479951,259.815014,261.891707,264.200052,268.024330,269.519566,269.940783,0.164550,0.164567,0.164587,0.165867,0.165874,0.165880,0.165882
4,345600.0,256.943673,259.080611,261.016618,263.251096,267.286315,269.131162,269.758407,0.164547,0.164561,0.164578,0.165867,0.165872,0.165878,0.165881


In [43]:
# Creating an instruction file suitable for the analysis
ins_data = pd.DataFrame(columns=sim_data.columns, index=sim_data.index)

# Adding the simulated variables [Temperature & Moisture] in the instruction file

# Temperature
for i, depth in enumerate(depths):
    for j, time in enumerate(times):
        # Column name = obsnme
        # (i + 1) - Signifies the start from the 2nd column
        ins_data.iloc[j+1, i+1] = f' !stemp_{depth}_{j}! ' 

# Moisture
for i, depth in enumerate(depths):
    for j, time in enumerate(times):
        # Column name = obsnme
        # (i + 1) - Signifies the start from the 2nd column
        ins_data.iloc[j+1, i+8] = f' !smois_{depth}_{j}! ' 

ins_data.head()

,time [s],point -0.04 temperature [K],point -0.1 temperature [K],point -0.2 temperature [K],point -0.4 temperature [K],point -0.8 temperature [K],point -1.2 temperature [K],point -1.6 temperature [K],point -0.04 saturation liquid,point -0.1 saturation liquid,point -0.2 saturation liquid,point -0.4 saturation liquid,point -0.8 saturation liquid,point -1.2 saturation liquid,point -1.6 saturation liquid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,!stemp_0.04_0!,!stemp_0.1_0!,!stemp_0.2_0!,!stemp_0.4_0!,!stemp_0.8_0!,!stemp_1.2_0!,!stemp_1.6_0!,!smois_0.04_0!,!smois_0.1_0!,!smois_0.2_0!,!smois_0.4_0!,!smois_0.8_0!,!smois_1.2_0!,!smois_1.6_0!
2,NaN,!stemp_0.04_1!,!stemp_0.1_1!,!stemp_0.2_1!,!stemp_0.4_1!,!stemp_0.8_1!,!stemp_1.2_1!,!stemp_1.6_1!,!smois_0.04_1!,!smois_0.1_1!,!smois_0.2_1!,!smois_0.4_1!,!smois_0.8_1!,!smois_1.2_1!,!smois_1.6_1!
3,NaN,!stemp_0.04_2!,!stemp_0.1_2!,!stemp_0.2_2!,!stemp_0.4_2!,!stemp_0.8_2!,!stemp_1.2_2!,!stemp_1.6_2!,!smois_0.04_2!,!smois_0.1_2!,!smois_0.2_2!,!smois_0.4_2!,!smois_0.8_2!,!smois_1.2_2!,!smois_1.6_2!
4,NaN,!stemp_0.04_3!,!stemp_0.1_3!,!stemp_0.2_3!,!stemp_0.4_3!,!stemp_0.8_3!,!stemp_1.2_3!,!stemp_1.6_3!,!smois_0.04_3!,!smois_0.1_3!,!smois_0.2_3!,!smois_0.4_3!,!smois_0.8_3!,!smois_1.2_3!,!smois_1.6_3!


In [44]:
# Removing the last character from the last column '~' to mimic the file sfr.csv.in
ins_data["point -1.6 saturation liquid"] = ins_data["point -1.6 saturation liquid"].str[:-1]
ins_data["point -1.6 saturation liquid"].tail()

1092     !smois_1.6_1091!
1093     !smois_1.6_1092!
1094     !smois_1.6_1093!
1095     !smois_1.6_1094!
1096     !smois_1.6_1095!
Name: point -1.6 saturation liquid, dtype: object

In [45]:
# Replacing the time vaiable with ~dum
ins_data['time [s]'] = f' !dum! '

# Dropping unnecessary row and column
#ins_data.drop(['time [s]'], axis=1, inplace=True)
ins_data.drop([0], axis=0, inplace=True)
ins_data.head()


,time [s],point -0.04 temperature [K],point -0.1 temperature [K],point -0.2 temperature [K],point -0.4 temperature [K],point -0.8 temperature [K],point -1.2 temperature [K],point -1.6 temperature [K],point -0.04 saturation liquid,point -0.1 saturation liquid,point -0.2 saturation liquid,point -0.4 saturation liquid,point -0.8 saturation liquid,point -1.2 saturation liquid,point -1.6 saturation liquid
1,!dum!,!stemp_0.04_0!,!stemp_0.1_0!,!stemp_0.2_0!,!stemp_0.4_0!,!stemp_0.8_0!,!stemp_1.2_0!,!stemp_1.6_0!,!smois_0.04_0!,!smois_0.1_0!,!smois_0.2_0!,!smois_0.4_0!,!smois_0.8_0!,!smois_1.2_0!,!smois_1.6_0!
2,!dum!,!stemp_0.04_1!,!stemp_0.1_1!,!stemp_0.2_1!,!stemp_0.4_1!,!stemp_0.8_1!,!stemp_1.2_1!,!stemp_1.6_1!,!smois_0.04_1!,!smois_0.1_1!,!smois_0.2_1!,!smois_0.4_1!,!smois_0.8_1!,!smois_1.2_1!,!smois_1.6_1!
3,!dum!,!stemp_0.04_2!,!stemp_0.1_2!,!stemp_0.2_2!,!stemp_0.4_2!,!stemp_0.8_2!,!stemp_1.2_2!,!stemp_1.6_2!,!smois_0.04_2!,!smois_0.1_2!,!smois_0.2_2!,!smois_0.4_2!,!smois_0.8_2!,!smois_1.2_2!,!smois_1.6_2!
4,!dum!,!stemp_0.04_3!,!stemp_0.1_3!,!stemp_0.2_3!,!stemp_0.4_3!,!stemp_0.8_3!,!stemp_1.2_3!,!stemp_1.6_3!,!smois_0.04_3!,!smois_0.1_3!,!smois_0.2_3!,!smois_0.4_3!,!smois_0.8_3!,!smois_1.2_3!,!smois_1.6_3!
5,!dum!,!stemp_0.04_4!,!stemp_0.1_4!,!stemp_0.2_4!,!stemp_0.4_4!,!stemp_0.8_4!,!stemp_1.2_4!,!stemp_1.6_4!,!smois_0.04_4!,!smois_0.1_4!,!smois_0.2_4!,!smois_0.4_4!,!smois_0.8_4!,!smois_1.2_4!,!smois_1.6_4!


In [46]:
#ins_data['l1'] = 'l1 ~'
# Adding an extra column to mimic sfr.csv.ins
ins_data.insert(0, 'l1', 'l1 ')

In [47]:
# Exporting the instruction file
ins_data.to_csv(f'{directory_name}/{file_name}_obs_data.dat.ins', header=False, index=False,sep=' ')

##### Dont forget to remove Quotes & add 'pif ~' string manually ;) (Dont know why the quotes are appearing as it did not in previous formats!
##### Also check if the spacing, indentation and rest of the content is correct by checking with the another instruction file (preferably by vs code?)

### Running pest in python:
1. Checks to be done

    a. TEMPCHEK - To check the template file
    
    b. INSCHEK - To check the instruction file
    
    c. Converting the file to version 1
    
    d. PESTCHEK - To check the pest control file
    
2. Running PEST (Not PEST++) with NOPTMAX = -1 (Wait until the results are displayed)
    
3. Running PWTADJ1 - pwtadj1 case1_v1.pst case2_v1_new.pst contribution

4. Running PEST++ for the redistributed weights


In [48]:
directory_name

'Case1_su_cv_start.demo'

In [49]:
os.getcwd()

'/home/rk/pestpp/pestpp/rk_model_final_4yrs_su'

In [57]:
# Change directory to required directory with simulation input files
os.chdir(f'{directory_name}')

In [51]:
os.getcwd()

'/home/rk/pestpp/pestpp/rk_model_final_4yrs_su/Case1_su_cv_start.demo'

1. Checks to be done:

a. Check ats input file - .xml file
    
b. TEMPCHEK - To check the template file
    
c. INSCHEK - To check the instruction file
    
d. Converting the file to version 1
    
e. PESTCHEK - To check the pest control file

In [52]:
# Testing subprocess
subprocess.check_output(['ls', '-l'])

b'total 4992\n-rw-rw-r-- 1 rk rk    3031 Jan 27 10:50 ats_modelcmd.py\n-rw-rw-r-- 1 rk rk     427 Jan 27 14:54 Case1_B_cv_cf.pst\n-rw-rw-r-- 1 rk rk 1359261 Jan 27 15:20 Case1_B_cv_cf_v1.pst\n-rw-rw-r-- 1 rk rk  882961 Jan 28 16:11 Case1_B_cv_cf_v1_rw.pst\n-rw-rw-r-- 1 rk rk      76 Jan 27 15:20 Case1_B_cv.insfile.csv\n-rw-rw-r-- 1 rk rk  337568 Jan 27 14:15 Case1_B_cv_obs_data.dat.obf\n-rw-rw-r-- 1 rk rk    1712 Jan 27 14:08 Case1_B_cv_par_data.csv\n-rw-rw-r-- 1 rk rk     995 Jan 27 14:07 Case1_B_cv_pargrp_data.csv\n-rw-rw-r-- 1 rk rk   58464 M\xc3\xa4r 17 10:53 Case1_B_cv_spinup_II_final.xml\n-rw-rw-r-- 1 rk rk      55 Jan 27 15:00 Case1_B_cv.tplfile.csv\n-rw-rw-r-- 1 rk rk  266269 Jan 28 15:33 Case1_B_cv_vm_obs_data.dat\n-rw-rw-r-- 1 rk rk   52420 Feb  1 09:12 Case1_B_cv_vm.xml\n-rw-rw-r-- 1 rk rk   54030 Jan 27 09:05 Case1_B_cv_woverbose.xml\n-rw-rw-r-- 1 rk rk   52424 Jan 27 15:50 Case1_B_cv.xml\n-rw-rw-r-- 1 rk rk     280 Jan 27 14:15 Case1_B_cv.xml.pmt\n-rw-rw-r-- 1 rk rk   5243

In [104]:
### Running ats within a temp folder

subprocess.run(['mkdir','test_ats.demo'], capture_output=True, text=True).stdout

''

In [22]:
os.chdir('test_ats.demo')

##### We need to also run the simulations on the cluster, therefore we have created a seperate file where the data access changes!

In [24]:
%%time
os.system(f'ats --xml_file=../{file_name}_vm.xml')

CPU times: user 985 µs, sys: 4.15 ms, total: 5.13 ms
Wall time: 1min 21s


0

Why is it not running? What does 34304 mean? - The ats file name was wrong! - See comment above

#### Check the outputs in the file and also the jupyter notebook terminal - weather it is running. If all is good you can start with the pestcheck!

In [53]:
os.chdir('..')

In [58]:
os.getcwd()

'/home/rk/pestpp/pestpp/rk_model_final_4yrs_su/Case1_su_cv_start.demo'

In [60]:
import re

In [61]:
file_name = 'Case1_su_cv' # Change
line_por_peat = 529 # Change (-1 of the line where porosity of peat is found)
line_por_mineral = 538 # Change (-1 of the line where porosity of mineral is found)


# 1. To find the porosity
filename = f'{file_name}.xml'

with open(f'{filename}') as oldfile:
            for line, content in enumerate(oldfile):
                if line == line_por_peat: # Line 603 (+1) has the porosity_peat 
                    poro_peat_line = str(content)
                    result = re.findall('\".*?\"', poro_peat_line)
                    poro_peat = float(result[2].replace('"',''))
                    print(poro_peat)
                elif line == line_por_mineral: # Line 612 (+1) has the porosity_mineral
                    poro_mineral_line = str(content)
                    result_2 = re.findall('\".*?\"', poro_mineral_line)
                    poro_mineral = float(result_2[2].replace('"',''))
                    print(poro_mineral)

0.63739496
0.48103077


In [62]:
%%time
### Check the ats_modelcmd.py file - Change the file name line of peat and mineral
os.system('python3 ats_modelcmd.py')
### Make sure that the observation file is created! - Testing in cmd is better 

CPU times: user 11.1 ms, sys: 6.38 ms, total: 17.5 ms
Wall time: 3min 48s


0

In [63]:
# Running tempchek
subprocess.run(['tempchek'], capture_output=True, text=True).stdout

' TEMPCHEK Version 17.3. Watermark Numerical Computing.\n\n TEMPCHEK is run using the command:\n\n    tempchek tempfile [modfile [parfile]]\n\n where\n\n    "tempfile" is a PEST template file,\n    "modfile" is an [optional] model input file to be written by TEMPCHEK, and\n    "parfile" is an [optional] parameter value file.\n\n'

In [64]:
# Running tempchek
subprocess.run(['tempchek',f'{file_name}.xml.tpl'], capture_output=True, text=True).stdout

' TEMPCHEK Version 17.3. Watermark Numerical Computing.\n\n Errors in file Case1_su_cv.xml.tpl ----->\n No errors encountered.\n\n 21 parameters identified in file Case1_su_cv.xml.tpl: these are listed in \n   file Case1_su_cv.xml.pmt.\n\n'

In [65]:
# Running inschek
subprocess.run(['inschek',f'{file_name}_obs_data.dat.ins'], capture_output=True, text=True).stdout

' INSCHEK Version 17.3. Watermark Numerical Computing.\n\n Errors in file Case1_su_cv_obs_data.dat.ins ----->\n No errors encountered.\n\n 15344 observations identified in file Case1_su_cv_obs_data.dat.ins: these \n   are listed in file Case1_su_cv_obs_data.dat.obf.\n\n'

In [66]:
### Converting the pest control file to version 1

In [67]:
import pyemu

In [68]:
directory_name

'Case1_su_cv_start.demo'

In [70]:
pst = pyemu.Pst(f"{file_name}_cf.pst")
pst.write(f"{file_name}_cf_v1.pst")

noptmax:-1, npar_adj:21, nnz_obs:15344


In [71]:
os.getcwd()

'/home/rk/pestpp/pestpp/rk_model_final_4yrs_su/Case1_su_cv_start.demo'

In [79]:
### Checking the pest control file once more

subprocess.run(['pestchek',f"{file_name}_cf_v1.pst"], capture_output=True, text=True).stdout

' PESTCHEK Version 17.3. Watermark Numerical Computing.\n\n Errors ----->\n No errors encountered.\n\n Warnings ----->\n It appears that the PEST control file contains a "rsi" section. PESTCHEK did \n   not check this. It can only be used by RSI_HP.\n NUMLAM is supplied as negative. This will be reset to positive. If Parallel \n   PEST or BEOPEST is used PARLAM will automatically be set to -9999.\n NOPTMAX provided as -1. No optimisation will be carried out; objective \n   function, residuals and statistics will be recorded for initial parameter \n   estimates only.\n MAXSING in the singular value decomposition section is greater than the \n   number of adjustable parameters.\n'

* If the error of DERCOM appears - Just remove 1 from the all the par_data lines!

#### Change the file name in 'ats_modelcmd.py' to pest file name

In [80]:
%%time
os.getcwd()

CPU times: user 25 µs, sys: 6 µs, total: 31 µs
Wall time: 34.8 µs


'/home/rk/pestpp/pestpp/rk_model_final_4yrs_su/Case1_su_cv_start.demo'

In [ ]:
%%time
### Running pest file

os.system(f'pest {file_name}_cf_v1.pst')
#subprocess.run(['pest',f'{file_name}_cf_v1.pst','&>out.log'], capture_output=True, text=True).stdout

### Reweighting strategy - PWTADJ1


In [78]:
subprocess.run(['pwtadj1'], capture_output=True, text=True).stdout

'\n PWTADJ1 version 17.3. Watermark Numerical Computing.\n\n PWTADJ1 is run using the command:\n\n     pwtadj1 casename pestoutfile contribution\n\n where\n\n     casename     is an existing PEST casename,\n     pestoutfile  is the name of a new PEST control file, and \n     contribution is the new objective function for each observation group.\n'

### Check the total objective function value in the .rec file and add in the cmd below:

 Objective function ----->

   - **Sum of squared weighted residuals (ie phi)                =  5.45014E-03**
   - Contribution to phi from observation group "temp"         =  2.01768E-03
   - Contribution to phi from observation group "mois"         =  3.43246E-03

In [30]:
obj_fn_value = 5.45014E-03 # Taken from .rec file

In [31]:
subprocess.run(['pwtadj1',f'{file_name}_cf_v1.pst',f'{file_name}_cf_v1_rw.pst',f'{obj_fn_value/2}'], capture_output=True, text=True).stdout

'\n PWTADJ1 version 17.3. Watermark Numerical Computing.\n\n - reading PEST control file Case1_B_cv_cf_v1.pst for first time...\n - file Case1_B_cv_cf_v1.pst read ok.\n\n - reading PEST run record file Case1_B_cv_cf_v1.rec...\n - file Case1_B_cv_cf_v1.rec read ok.\n\n - re-reading file Case1_B_cv_cf_v1.pst and writing file Case1_B_cv_cf_v1_rw.pst...\n - file Case1_B_cv_cf_v1.pst read ok.\n - file Case1_B_cv_cf_v1_rw.pst written ok.\n'

In [32]:
# Removing the out.log file since it's file size is large
subprocess.run(['rm','-rf','out.log'], capture_output=True, text=True).stdout

''

In [45]:
os.chdir(f'..')

In [48]:
os.getcwd()

'/home/rk/pestpp/pestpp/rk_model_final_4yrs'

In [49]:
file_name

'Case1_B_cv'

In [50]:
directory_name_rw = f'{file_name}_rw.demo'

In [51]:
directory_name

'Case1_B_cv_start.demo'

In [52]:
# Copying the files to a new folder
shutil.copytree(directory_name, directory_name_rw)

'Case1_B_cv_rw.demo'

In [53]:
os.chdir(f'{directory_name_rw}')

In [54]:
cf_name = f'{file_name}_cf_v1' # Control file name befor reweighting

In [55]:
# Removing unnecessary files to provide space for the simulation
# Check once more if all the files have been removed properly
os.system(f'rm {cf_name}.drf {cf_name}.jac {cf_name}.jst {cf_name}.jco {cf_name}.mtt {cf_name}.par {cf_name}.rec {cf_name}.rei {cf_name}.res {cf_name}.sen {cf_name}.seo {cf_name}.svd {cf_name}.rst jacob.runs')

256

In [56]:
file_name

'Case1_B_cv'

In [57]:
%%time
# Running the new re-weighted pest file
os.system(f'pestpp-glm {file_name}_cf_v1_rw.pst')
# Prefer running it on the terminal?

CPU times: user 146 ms, sys: 309 ms, total: 455 ms
Wall time: 2h 49min 8s


0